<a href="https://colab.research.google.com/github/DemianDeveloper/Machine_Learning/blob/main/XRP_logistical_regression_price_predictable_FGI%2BRSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*прогноз движения XRP на основе индексов FGI(индекс страха и жадности) и RSI. расчёт сделан опираясь на последние 1300 дней*

*выполнено путём создания модели машинного обучения (логистическая регрессия)*






# ОПИСАНИЕ ИСПОЛЬЗУЕМЫХ ФУНКЦИЙ



---








Загрузка данных о Fear and Greed Index (FGI):
API индекса FGI берём от сюда https://alternative.me/crypto/fear-and-greed-index/


Функция parse_fgi(N) используется для загрузки данных о FGI с помощью API и возвращает список значений индекса.
Параметр N указывает количество последних значений FGI для загрузки.

Установка и подключение Ta-Lib:
С помощью команд !wget, !tar, !./configure, !make, !make install устанавливаются и подключаются библиотеки Ta-Lib.
Получение и обработка исторических данных о ценах:

Функция parse_dates(N) используется для генерации списка дат на основе количества дней N.
Функция parse_prices(coin, N) загружает исторические данные о ценах для указанной криптовалюты coin за последние N дней с помощью библиотеки yfinance.

Вычисление индикаторов:
Функция indicators(df, N) добавляет в DataFrame индикаторы, такие как Relative Strength Index (RSI) и Simple Moving Average (SMA), а также значения FGI.
Построение корреляционной матрицы:

Функция corr_matrix(df) строит корреляционную матрицу между ценами закрытия, RSI, FGI и SMA с помощью библиотеки seaborn.
Построение графиков:

Графики цен закрытия и индикаторов RSI, FGI и SMA строятся с использованием библиотеки matplotlib.

Предварительная обработка данных:
Функция preprocessing(df) производит предварительную обработку данных, такую как удаление нулевых значений и разделение данных на признаки (x), целевую переменную (y) и цены закрытия (p).

# ПОЯСНЕНИЯ РАБОТЫ МОДЕЛИ
---



Обучение модели:
Создается модель машинного обучения (логистическая регрессия) и обучается на данных при помощи функции ML(clf, X_train, X_test, Y_train, Y_test).

Оценка результатов:
Вычисляется точность предсказаний модели.

Анализ сигналов и принятие решений:
Сигналы от модели преобразуются в решения о покупке (1), продаже (-1) или удержании позиции (0).

Построение графика с маркерами решений:
График цен закрытия с маркерами для принятых решений (покупка - зеленый маркер, продажа - красный маркер) строится с использованием matplotlib.
Каждая часть кода выполняет определенную задачу в процессе анализа данных и торговли на основе модели машинного обучения.

In [3]:
import requests

In [2]:
N = 1300 # Устанавливаем количество дней для загрузки данных FGI

In [4]:
# создаём функцию запроса FGI
def parse_fgi(N):
  url = 'https://api.alternative.me/fng/?format=csv&date_format=us'
  parameters = {
      'limit': N,
      'format': 'json'
  }

  response = requests.get(url, params=parameters)
  data = response.json()['data']
  fgi_values = [float(i['value']) for i in data][::-1]

  return fgi_values

fgi_values = parse_fgi(N)# Получаем значения FGI

In [ ]:
# Устанавливаем и подключаем Ta-Lib

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [6]:
from datetime import datetime, timedelta
import pandas as pd
import talib
import numpy as np
from talib import abstract
import yfinance as yf


In [7]:
# Функция для генерации дат от даты N дней назад до текущей даты.

def parse_dates(N):#объявление функции с именем parse_dates, которая принимает один аргумент N.
  end_date = datetime.today()#Создает переменную end_date, содержащую текущую дату и время.
  start_date = end_date - timedelta(days=N)# Вычитает из текущей даты N дней и сохраняет результат в переменной start_date. Таким образом, start_date будет N дней назад от текущей даты.
  dates = pd.date_range(start=start_date, end=end_date)#Создает ряд дат в заданном диапазоне от start_date до end_date с помощью функции pd.date_range.
  us_dates = dates.strftime('%Y-%m-%d').tolist()#Преобразует каждую дату из объекта dates в строку в формате %Y-%m-%d (год-месяц-день) и сохраняет их в виде списка строк с помощью метода strftime. Затем преобразует полученный объект в список с помощью метода tolist().

  return us_dates



In [ ]:
# Функция для загрузки исторических цен

def parse_prices(coin, N):
    period = parse_dates(N) # Вызывает функцию parse_dates с аргументом N для создания периода времени (списка дат) для запроса данных.
    end_time = period[-1] # end_time = period[-1]: Устанавливает переменную end_time равной последней дате из периода времени period.
    start_time = period[0] # Устанавливает переменную start_time равной первой дате из периода времени period.
    data = yf.download(coin+'-USD', start_time, end_time)#Загружает исторические данные о ценах с использованием библиотеки yfinance.
    data.rename(columns = {'Open': 'open',#Переименовывает столбцы данных. Здесь мы переименовываем столбцы 'Open', 'High', 'Close' и 'Volume' соответственно в 'open', 'high', 'close' и 'volume'. inplace=True означает, что изменения применяются к исходному DataFrame.
                         'High': 'high',
                         'Close': 'close',
                         'Volume': 'volume'}, inplace =True)
    return data

# Получаем исторические данные о ценах для XRP
data_prices = parse_prices('XRP', N)
print(data_prices)


In [ ]:
# Функция для добавления индикаторов

def indicators(df, N):# объявление функции с именем indicators, которая принимает два аргумента: df (DataFrame с данными о ценах) и N (количество дней).
  df['rsi'] = abstract.RSI(df)# Добавляет столбец 'rsi' к DataFrame df, содержащий значения RSI. Функция abstract.RSI() вызывается для вычисления RSI на основе данных о ценах.
  df['sma'] = abstract.SMA(df)
  df['fgi'] = parse_fgi(N)
  return df

df = indicators(data_prices, N)# Добавляем индикаторы к данным о ценах
df

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Функция для построения корреляционной матрицы
def corr_matrix(df):
    # Выбираем только нужные столбцы для анализа корреляции
    df = df[['close', 'rsi', 'fgi', 'sma']]

    # Вычисляем корреляционную матрицу с использованием метода Пирсона
    corr_df = df.corr(method='pearson')

    # Создаем объект "фигура" для рисования
    plt.figure(figsize=(18, 10))

    # Строим тепловую карту корреляционной матрицы с аннотациями
    sns.heatmap(corr_df, cmap='bwr', annot=True)

    # Показываем график
    plt.show()

# Вызываем функцию corr_matrix для DataFrame df
corr_matrix(df)


In [ ]:
# Создаем объект "фигура" и наборы осей с указанным количеством строк и столбцов
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

# Строим графики для столбцов 'close' и 'sma' на первой оси
df[['close', 'sma']].plot(ax=axes[0], title='Close Prices & MA', xlabel='Date', ylabel='Value')

# Строим графики для столбцов 'fgi' и 'rsi' на второй оси
df[['fgi', 'rsi']].plot(ax=axes[1], title='RSI & FGI', xlabel='Date')

# Показываем графики
plt.show()


In [ ]:
# Создаем массив y, где 1 обозначает повышение цены, а 0 - понижение
y = np.where(df['close'].shift(-1) > df['close'], 1, 0)

# Добавляем столбец 'y' к DataFrame
df['y'] = y

# Создаем столбец 'p', который содержит значения цены закрытия
df['p'] = df['close']

# Выводим столбцы 'p' и 'y'
df[['p', 'y']]


In [ ]:
# Функция для предварительной обработки данных
def preprocessing(df):
    # Удаляем строки с пропущенными значениями
    df.dropna(inplace=True)

    # Определяем переменную x, содержащую признаки для обучения модели
    x = df[['rsi', 'fgi', 'sma']]

    # Определяем переменную y, содержащую целевую переменную
    y = df[['y']]

    # Определяем переменную p, содержащую цены закрытия
    p = df[['p']]

    return x, y, p

# Вызываем функцию предварительной обработки данных
x, y, p = preprocessing(df)

# Выводим переменные x и y
x, y


In [ ]:
# Определяем процент данных для обучения
percent_for_train = 0.7

# Вычисляем количество строк, которые будут использоваться для обучения
split = int(percent_for_train * len(x))

# Разделяем данные на обучающие и тестовые
X_train = x[:split]
Y_train = y[:split]
p_train = p[:split]

X_test = x[:split]
Y_test = y[:split]
p_test = p[:split]

# Выводим обучающие и тестовые данные
[X_test, Y_test]


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [17]:
model = LogisticRegression()

In [18]:
# Определяем функцию для обучения и оценки модели
def ML(clf, X_train, X_test, Y_train, Y_test):
    # Обучаем модель
    clf.fit(X_train, Y_train)

    # Получаем прогнозы
    preds = clf.predict(X_test)

    # Вычисляем метрику точности
    acc_score = accuracy_score(Y_test, preds)

    return preds, acc_score

# Обучаем модель и оцениваем её точность
preds, accuracy_score = ML(model, X_train, X_test, Y_train, Y_test)
accuracy_score


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5230596175478065

1 - цена пойдёт вверх
0 - цена пойдёт вниз

однако далее в коде преобразуем логику в следующую и теперь такая логика№:
0 - удержание позиции
1 - покупаем
-1 - продаём

In [24]:
# Генерируем сигналы на основе прогнозов модели
signals = []

# Проходим по каждому прогнозу
for pred in preds:
    # Если прогноз равен 0, добавляем -1 в список сигналов
    if pred == 0:
        signals.append(-1)
    # Если прогноз не равен 0, добавляем 1 в список сигналов
    else:
        signals.append(1)

# Инициализируем начальную позицию
position = 0

# Создаем список решений
decisions = []

# Проходим по каждому сигналу
for signal in signals:
    # Если сигнал равен -1
    if signal == -1:
        # Если текущая позиция также равна -1, добавляем 0 в список решений
        if position == -1:
            decisions.append(0)
        # Если текущая позиция не равна -1, добавляем -1 в список решений и обновляем текущую позицию
        else:
            decisions.append(signal)
            position = signal
    # Если сигнал не равен -1
    else:
        # Если текущая позиция равна 1
        if position == 1:
            decision = signal
        # Если текущая позиция не равна 1, добавляем сигнал в список решений и обновляем текущую позицию
        else:
            decisions.append(signal)
            position = signal

# Подсчитываем количество положительных решений
count = decisions.count(1)
count


32

In [26]:
prices = p_test['p'].tolist()

# Проверяем длины списков
print("Длина списка решений:", len(decisions))
print("Длина списка цен:", len(prices))
print("Длина списка прогнозов:", len(preds))
print("Длина списка сигналов:", len(signals))

# Создаем DataFrame только если длины всех списков одинаковы
if len(decisions) == len(prices) == len(preds) == len(signals):
    # Создаем DataFrame с решениями, ценами, прогнозами и сигналами
    back_test_df = pd.DataFrame({'decision': decisions, 'price': prices, 'signal': preds, 'signals0': signals}, index=x_test.index)
    # Получаем даты из индекса
    dates = x_test.index.tolist()
    # Добавляем столбец даты и устанавливаем его как индекс
    back_test_df['date'] = dates
    back_test_df.set_index('date', inplace=True)
    print("DataFrame успешно создан.")
else:
    print("Длины списков не совпадают, невозможно создать DataFrame.")



Длина списка решений: 417
Длина списка цен: 889
Длина списка прогнозов: 889
Длина списка сигналов: 889
Длины списков не совпадают, невозможно создать DataFrame.


In [29]:
prices = p_test['p'].tolist()

# Используем только первые 417 элементов из списков с длиной 889
prices = prices[:417]
preds = preds[:417]
signals = signals[:417]

# Создаем DataFrame на основе первых 417 элементов из списков
back_test_df = pd.DataFrame({'decision': decisions[:417], 'price': prices, 'signal': preds[:417], 'signals0': signals[:417]}, index=X_test.index[:417])
# Добавляем столбец 'date' с индексом и устанавливаем его как индекс DataFrame
back_test_df['date'] = back_test_df.index
back_test_df.set_index('date', inplace=True)



In [ ]:
# Сбрасываем индекс и переименовываем столбец 'index' в 'date'
back_test_df.reset_index(inplace=True)
back_test_df.rename(columns={'index': 'date'}, inplace=True)#Переименовывает столбец индекса в 'date'.

# Построение графика цены актива с маркерами решений
plt.figure(figsize=(10, 6))
plt.plot(back_test_df['date'], back_test_df['price'], linestyle='-')# Строит линейный график цены актива по времени.

for index, row in back_test_df.iterrows():
    if row['decision'] == -1: #сли решение равно -1 (продажа), то
        plt.plot(row['date'], row['price'], marker='o', color='red', markersize=5)#Рисует красный кружок маркера на графике в точке, где было принято решение о продаже.
    elif row['decision'] == 1:
        plt.plot(row['date'], row['price'], marker='o', color='green', markersize=5)

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Price with Decision Markers')
plt.grid(True)

plt.show()

